
# **Measurements**
 
## **Introduction**
 
This is the third and last script in the processing pipeline for IMC data.

The goal is to extract measurements from the multichannel images generated in the first script using the cell masks generated in the second script. 
This is performed using functions from the `steinbock` package. 
Documentation can be found here: https://bodenmillergroup.github.io/steinbock/latest/cli/measurement.
 
The following measurements are performed: 

**Measure intensities**  
- Average marker intensities for single cells.
 
**Measure region properties**  
- Cell-level spatial measurements..

 In case one of the two data files is not generated (for instance due to a missing mask), 
 the corresponding files in other data folders are deleted at the end of this script.


## **Configuration**
 
### **Import packages**

In [1]:
import logging
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import sys

from scipy import ndimage as ndi
from skimage import measure
from skimage.segmentation import expand_labels
from skimage.util import invert

from steinbock import io
from steinbock.measurement import intensities, neighbors, regionprops

In [2]:
logger = logging.getLogger(__name__)
print(sys.path)
print(sys.executable)
base_dir = Path("/home/processing/")

['/home/T1D_preprocessing', '/opt/conda/lib/python39.zip', '/opt/conda/lib/python3.9', '/opt/conda/lib/python3.9/lib-dynload', '', '/opt/conda/lib/python3.9/site-packages']
/opt/conda/bin/python


### **Load directories and panels**
 
Paths to input and output folders as well as antibody panels were exported by the first script (`01_Preprocessing.ipynb`). Here they are imported again.


In [3]:
%ls

01_Preprocessing.ipynb*     03_Measurements.ipynb*
02_CellSegmentation.ipynb*  README.md*


In [4]:
with open(base_dir / "variables/folders.txt", "rb") as handle:
    folders = pickle.loads(handle.read())
folders

with open(base_dir / "variables/panels.txt", "rb") as handle:
    panels = pickle.loads(handle.read())

for panel_name, panel in panels.items():
    print(panel_name, "\n", panel.head())
panel_names = list(panels.keys())

Uncompressed 
    channel  metal                     name    antibody_clone  keep  deepcell  \
0        0  In113               Histone H3              D1H2     1       1.0   
1        1  In115                      SMA               1A4     1       2.0   
5        5  Pr141                  insulin             C27C9     1       0.0   
6        6  Nd143                     CD44               IM7     1       2.0   
7        7  Nd144  Prohormone Convertase 2  Polyclonal _ PC2     1       0.0   

   clustering  dimred shortname  
0           0       0        H3  
1           1       1       SMA  
5           1       1       INS  
6           1       1      CD44  
7           1       1     PCSK2  
Compressed 
    channel  metal                     name  antibody_clone  keep  deepcell  \
0        0  In113               Histone H3            D1H2     1       1.0   
1        1  In115                      SMA             1A4     1       2.0   
5        5  Nd143                 CD44_GCG      IM7_D

#### **Create output directories**

In [5]:
# Cell data
measurements = ["intensities", "regionprops", "neighbors"]

for panel_name in panel_names:
    output_dir_cells = folders["data_cells"] / panel_name
    output_dir_cells.mkdir(exist_ok=True)
    
    for meas_type in measurements:
        meas_dir_cells = output_dir_cells / meas_type
        meas_dir_cells.mkdir(exist_ok=True)

#### **Select the cell segmentation type to use**

In [6]:
segmentation_type = "whole-cell"

## **Measure intensities**
 
Here, the mean marker expression over the cell area is measured.

Full documentation: https://bodenmillergroup.github.io/steinbock/latest/cli/measurement/#object-intensities.


### **Single cell intensities**

In [7]:
for panel_name in panel_names:
    img_subdir = folders["img"] / panel_name
    masks_subdir = folders["masks_cells"] / panel_name / segmentation_type
    intensities_dir = folders["data_cells"] / panel_name / "intensities"
    
    for img_path, mask_path, intens in intensities.try_measure_intensities_from_disk(
        img_files = io.list_image_files(img_subdir),
        mask_files = io.list_image_files(masks_subdir),
        channel_names = panels[panel_name]["name"],
        intensity_aggregation = intensities.IntensityAggregation.MEAN
    ):
        intensities_file = img_path.name.replace('.tiff', '.csv')
        pd.DataFrame.to_csv(intens, Path(intensities_dir) / intensities_file)

## **Measure region properties**

Documentation for region properties measurements: https://bodenmillergroup.github.io/steinbock/latest/cli/measurement/#region-properties.
 
### **Properties to measure**
 
For a full list of measurable properties, refer to https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops.


In [8]:
skimage_regionprops = [
        "area",
        "centroid",
        "major_axis_length",
        "minor_axis_length",
        "eccentricity",
    ]

### **Cell-level spatial measurements**
 
Measurement of spatial properties for single cells.  

In [9]:
for panel_name in panel_names:
    img_subdir = folders["img"] / panel_name
    cell_masks_subdir = folders["masks_cells"] / panel_name / segmentation_type
    regions_dir = folders["data_cells"] / panel_name / "regionprops"
    
    img_files = io.list_image_files(img_subdir)
    
    for img_file in img_files:
        try:
            # Load images and masks
            cell_mask_file = cell_masks_subdir / img_file.name

            if (cell_mask_file.exists()):
                img = io.read_image(img_file)
                cell_mask = io.read_mask(cell_mask_file)

                # Measure cell-level region props
                region_props = regionprops.measure_regionprops(img, cell_mask, skimage_regionprops)

                # Save measurements as CSV files
                regprop_file = img_file.name.replace(".tiff", ".csv")
                pd.DataFrame.to_csv(region_props, Path(regions_dir) / regprop_file)
            
        except:
            logger.exception(f"Error measuring regionprops in {img_file}")    

## **Catch unmatched data files**
 
### **Flag and delete unmatched data files**
For each image, three data files should be generated corresponding to intensities, and region props.
If for one image one of these files is missing, the other ones are removed in order to avoid conflicts when importing data into R. Data files that do not have a matching file in every measurement folders are deleted.

In [10]:
delete_unmatched_files = True

for panel_name in panel_names:
    missing = set()
    
    # List intensity files
    intensity_dir = folders["data_cells"] / panel_name / "intensities"
    intensity_files = Path(intensity_dir).rglob("[!.]*.csv")
    intensity_files = frozenset(file.name for file in intensity_files)

    # Find matched data files in the other cell data folders
    for meas_type in ["regionprops"]:
        cur_dir = folders["data_cells"] / panel_name / meas_type
        cur_files = set([file.name for file in Path.iterdir(cur_dir)])

        missing.add(frozenset(intensity_files.difference(cur_files)))
        missing.add(frozenset(cur_files.difference(intensity_files)))
        
    # Print out all missing images
    missing = [list(x) for x in missing]
    missing = [x for xs in missing for x in xs]
    print("Images with missing corresponding files:\n", missing)
    
    # Delete unmatched data files
    if delete_unmatched_files:
        unmatched_files = []
        for meas_type in ["intensities", "regionprops"]:
            cur_dir = folders["data_cells"] / panel_name / meas_type
            unmatched_files.extend([cur_dir / file for file in missing])

        print("\nDeleted files:")
        for file in unmatched_files:
            if file.is_file():
                print(file)
                Path.unlink(file, missing_ok=True)


Images with missing corresponding files:
 []

Deleted files:
Images with missing corresponding files:
 []

Deleted files:



## **Next step**
 
This notebook is the last one in this processing pipeline. The next step is to load the measurements extracted here in R for data analysis. All data analysis for the current project is performed using the analysis repository.